<a href="https://colab.research.google.com/github/joel-winterton/Contagion/blob/main/Full%20simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [147]:
import pandas as pd
import numpy as np

In [148]:
processed_data_path = "/content/drive/MyDrive/BBCPandemic/Processed"

## Within patch transition function

In [149]:
def make_transition_rate_fn(infectious_profile, reproductive_number, contact_matrix, group_sizes):
  """
  Precalculates parameters and returns function to calculate transition rates.
  """
  # Calculate A from contact matrix
  # multiply jth column by jth population and divide ith row by ith population using diagonal matrices
  A_hat = np.matmul(np.diag(1/group_sizes),np.matmul(contact_matrix , np.diag(group_sizes)))
  # Divide through by maximum eigenvalue
  A = A_hat/np.linalg.eigh(A_hat)[0][-1]
  tau_max = len(infectious_profile) - 1
  number_of_groups = len(group_sizes)
  infectious_profile_matrix = np.broadcast_to(infectious_profile[1:]/infectious_profile.sum(), (number_of_groups, len(infectious_profile)-1))

  def transition_rate_fn(t, state):
    """
    Calculates the transition rate for each group at time t in days given the state before t.
    Returns array where ith element is ith force of infection.
    """
    index = t + 5
    S,I = state
    infections = np.array(I[:,index-tau_max:index])
    transition_values = infectious_profile_matrix*np.matmul(A, infections)
    return reproductive_number*transition_values.sum(axis=1)

  return transition_rate_fn

# Between patch accounting

In [150]:
class Infections:
  def __init__(self, t_max: int, patch_ages, age_contact_matrix,distance_matrix, rural_urban, rural_proportions, urban_proportions, beta_profile, R_nought, epsilon):
    """
    t_max: Maximum number of days to run simulation for.
    patch_ages: Number of people in each age group for each patch, ndarray(patch, age_group).
    age_contact_matrix: Average contact matrix between age groups (called B for BBC in Contagion paper), ndarray(age_group, age_group).
    distance_matrix: Matrix containing distance (nearest km) between each patch, ndarray(patch, patch).
    rural_urban: Rural/urban classification for each patch, ndarray(patch).
    rural_proportions: Array containing proportion of time someone in rural patch spends d km from home, nparray(distance).
    urban_proportions: Same as rural_proprotions but for urban patches.
    beta_profile: Beta value for each day of infection, starting at 0, ndarray(length_of_infection).
    R_nought: Basic reproductive number, float.
    epsilon: Normalising constant for force of infection, float.
    """
    self.patch_ages = patch_ages
    self.beta_profile = beta_profile/beta_profile.sum()
    self.R_nought = R_nought
    self.t_max = t_max
    self.age_contact_matrix = age_contact_matrix
    self.urban_proportions = urban_proportions
    self.rural_proportions = rural_proportions
    self.rural_urban= rural_urban
    self.distance_matrix = distance_matrix
    self.epsilon = epsilon
    # Derived properties
    self.number_of_patches = patch_ages.shape[0]
    self.number_of_age_groups = patch_ages.shape[1]
    self.incidence = np.zeros(shape=(self.number_of_patches, self.t_max+6, self.number_of_age_groups))
    self.susceptibles = np.zeros(shape=(self.number_of_patches, self.t_max+6, self.number_of_age_groups))
    self.onset_times = np.full(shape=(self.number_of_patches), fill_value=self.t_max+2, dtype=int)

  def simulate_patch(self, j:int, tau:int)->None:
    """
    Run deterministic model for patch j,
    and propogate simulation into susceptibles and incidence object,
    starting at index tau.
    """
    number_of_days = self.t_max - tau
    self.onset_times[j] = tau
    # first five entries are before t=0
    I = np.array([[0.002, 0.002, 0.002, 0.002, 0.002, 0.00]+[-1.]*number_of_days]*self.number_of_age_groups)
    S = np.array([[0., 0., 0., 0., 0., 0.99]+[-1.]*number_of_days]*self.number_of_age_groups)

    trans_fn = make_transition_rate_fn(self.beta_profile, self.R_nought, self.age_contact_matrix, self.patch_ages[j])
    for t in range(number_of_days):
      index = t+5
      forces_of_infection = trans_fn(t, (S,I))
      probability_of_infection = 1 - np.exp(-forces_of_infection)
      S[:,index+1] = S[:,index]*(1-probability_of_infection)
      I[:,index+1] = S[:,index]*probability_of_infection
    # Transpose
    self.incidence[j,tau:,:] = I.T
    self.susceptibles[j,tau:,:] = S.T

  def get_active_patches(self, t):
    """
    Returns indices of patches that have been infected by time t.
    """
    return np.argwhere(self.onset_times<=t)

  def get_inactive_patches(self, t):
    """
    Returns indices of patches have not been infected up to time t.
    """
    return np.argwhere(self.onset_times>t)

  def patch_effective_prevalence(self, j:int,t:int)->float:
    """
    Returns phi_j(t), the relative force of infection patch k exerts at t.
    """
    tau_indices = np.arange(1, len(self.beta_profile))
    incidence_slices = self.incidence[j, t - tau_indices[:, np.newaxis], :]
    weighted_incidence = (incidence_slices * self.patch_ages[j, :]).sum(axis=-1)
    return np.dot(self.beta_profile[1:], weighted_incidence)

  def force_of_infection(self, j:int, t:int)->float:
    """
    Returns lambda_k(t), the force of infection on patch k at t.
    """
    patch_population_size = self.patch_ages[j,:].sum()
    active_patches = self.get_active_patches(t)
    patch_times = t - self.onset_times[active_patches]

    vectorised_patch_prevalence = np.vectorize(lambda patch,time: self.patch_effective_prevalence(patch, time))
    vectorised_proportions = np.vectorize(lambda l: self.distance_proportion(j, l))

    patch_forces = vectorised_patch_prevalence(active_patches, patch_times)
    patch_proportions = vectorised_proportions(active_patches)
    force_of_infection = self.epsilon*patch_population_size* (patch_forces*patch_proportions).sum()
    return force_of_infection

  def distance_proportion(self, j:int, l: int)->float:
    """
    Returns F_ru(j)(d_{jl}), the proportion of visits from rural/urban (depending on ru(j)) that are d_jl km from the origin.
    """
    d = self.distance_matrix[j,l]
    if self.rural_urban[j]:
      return self.rural_proportions[d]
    else:
      return self.urban_proportions[d]

  def simulate_between_patch(self, seed_location, seed_time):
    """
    This brings everything together by simulating between patch.
    """
    # Seed
    self.simulate_patch(j=seed_location, tau=seed_time)
    # Move forward in time
    for t in range(self.t_max - seed_time):
      # Calculate infection forces
      patches = self.get_inactive_patches(t)
      vectorised_force_of_infection = np.vectorize(lambda j: self.force_of_infection(j, t))
      forces_of_infection = vectorised_force_of_infection(patches).ravel()
      # Probability to seed in that location
      probability_of_infection = 1 - np.exp(-forces_of_infection)
      # Seed patches with probability from above
      unif_samples = np.random.default_rng().uniform(size=len(patches))
      seed_patches = unif_samples<probability_of_infection
      indexes_to_seed = patches[seed_patches]
      for index in indexes_to_seed:
        self.simulate_patch(j=index[0], tau=t)




# Bring in the data


In [151]:
# Maximum time
t_max = 250

# Patch population by ages (and patch id to LDA code)
patch_ages = pd.read_csv(processed_data_path+"/lad_population_by_age.csv", header=None, skiprows=1)
patch_id_to_lad = patch_ages[0].values
patch_ages.drop(columns=0, inplace=True)
patch_ages = patch_ages.values

# Age contact matrix
age_contact_matrix = pd.read_csv(processed_data_path+"/contact_polymod.csv", header=None, skiprows=1)
age_contact_matrix.drop(columns=0, inplace=True)
age_contact_matrix = age_contact_matrix.values

# Patch distances
distance_matrix = pd.read_csv(processed_data_path+"/centroid_distances.csv", header=None, skiprows=1)
distance_matrix.drop(columns=0, inplace=True)
distance_matrix = distance_matrix.values

# Rural/Urban classification
rural_urban = pd.read_csv(processed_data_path+"/rural_data.csv")
rural_urban = rural_urban['rural'].values

# Flux proportions
urban_proportions = pd.read_csv(processed_data_path+"/urban_proportions.csv")
urban_proportions.set_index('Unnamed: 0', inplace=True)
urban_proportions = np.array([urban_proportions.loc[i,'count'] if i in urban_proportions.index else 0 for i in range(distance_matrix.max()+1)])

rural_proportions = pd.read_csv(processed_data_path+"/rural_proportions.csv")
rural_proportions.set_index('Unnamed: 0', inplace=True)
rural_proportions = np.array([rural_proportions.loc[i,'count'] if i in rural_proportions.index else 0 for i in range(distance_matrix.max()+1)])


# Infectivity parameters
beta_profile = np.array([0., 0., 1.6, 0.8, 0.2, 0.2])
R_nought = 1.8
epsilon = 10

In [152]:
patch_ages.shape

(326, 13)

# Whack it all together

In [153]:
infections = Infections(t_max=t_max,
                        patch_ages=patch_ages,
                        age_contact_matrix=age_contact_matrix,
                        distance_matrix = distance_matrix,
                        rural_urban = rural_urban,
                        rural_proportions=rural_proportions,
                        urban_proportions=urban_proportions,
                        beta_profile=beta_profile,
                        R_nought=R_nought,
                        epsilon=epsilon)

infections.simulate_between_patch(seed_location=0, seed_time=0)

In [154]:
print(infections.onset_times)

[  0   2   2   2   2   6   6   6   6   6   6   6   6   2   6   7   7   6
  16  14   6  30  30  30  22  70  70  38  38  22  14  16  17  18  18  18
  18  18  18  18  18  14  18  18  34  35   2   2   6   6   7 126 252  22
  14  16  16  18  18  18  16  14  14  14  16   6   6   6   6   2   6   6
   6   6   6   6   6   6   7  62  62  62  62  70  65  70  70  34  34  34
  35  38  38  18  19  18  18  18  18  18  18  18  18  18  18  18  17  18
  18  16  16  16  16  16  16  16  18  18  19  18  19  18  18  34  18  19
  18  18  16  16  18  16  18  16  18  18  18  18  18  18  19  18  18  18
  18  18  19  18  18   6   6   6   6   6   6   6   6   6   6   6   6  14
   7  14  14   7   7   7  14   6   6   6   7  14   6  14  16  30  14  14
  16  16  14  14  14  14  14  14  14   2   2   2   2   2   2   2   6   6
   6   6   6   6   6  16  16  16  16  16  34  34  34  34  46   7   7   7
   6   7  14   6   8  18  16  19  16  16  30  30  18  18  18  18  18  18
  18  18  18  18  18   7   7  14  10  14  18  18  1